In [1]:
!pip3 install numpy pandas scikit-learn catboost langchain-openai langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 2.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.7 MB/s eta 0:00:00


In [19]:
import numpy as np
import pandas as pd
import os

from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [20]:
train_df = shuffle(pd.read_csv('Training.csv').iloc[:, :-1])
test_df = pd.read_csv('Testing.csv')

train_df.reset_index().head(5)

,index,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,923,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hypertension
1,3747,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chicken pox
2,535,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Cervical spondylosis
3,1017,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,hepatitis A
4,48,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Drug Reaction


In [21]:
train_x = train_df.iloc[:, :-1]
train_y = train_df.iloc[:, -1]

test_x = test_df.iloc[:, :-1]
test_y = test_df.iloc[:, -1]

# Targets

In [22]:
print(test_y.unique().tolist())

['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis', 'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes ', 'Gastroenteritis', 'Bronchial Asthma', 'Hypertension ', 'Migraine', 'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice', 'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A', 'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E', 'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia', 'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins', 'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia', 'Osteoarthristis', 'Arthritis', '(vertigo) Paroymsal  Positional Vertigo', 'Acne', 'Urinary tract infection', 'Psoriasis', 'Impetigo']


In [29]:
model = CatBoostClassifier(iterations=50, learning_rate=0.1, depth=2, loss_function='MultiClass', verbose=False)
model.fit(train_x, train_y)

predictions = model.predict(test_x)
accuracy = accuracy_score(test_y, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9761904761904762


In [30]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key="sk-proj-JJmAyAgkHoicCDTguLSMT3BlbkFJESclE9IKPcNuTUMD1KVc") # type: ignore

In [31]:
model.save_model('hackap.cbm')

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert in medical science."),
        ("human", "Please provide general prescriptions for {user_input}"),
    ]
)

chain = prompt | llm

In [33]:
def expert_advice():
    disease=model.predict(train_x.head(1))
    print("Judging from symptoms, you are most likely facing:",disease.tolist())
    print()
    response=chain.invoke({"user_input": disease})
    print(response.content)

In [34]:
expert_advice()

Judging from symptoms, you are most likely facing: [['Hypertension ']]

For hypertension, also known as high blood pressure, the general treatment approach includes lifestyle modifications and medications. Here are some general prescriptions for managing hypertension:

1. Lifestyle modifications:
   - Maintain a healthy weight: Losing weight if overweight or obese can help lower blood pressure.
   - Follow a healthy diet: Eat a diet rich in fruits, vegetables, whole grains, and lean proteins. Limit sodium intake and avoid processed foods.
   - Exercise regularly: Aim for at least 150 minutes of moderate-intensity exercise per week.
   - Limit alcohol intake: Limit alcohol consumption to moderate levels (1 drink per day for women, 2 drinks per day for men).
   - Quit smoking: Smoking can raise blood pressure, so quitting smoking is important for managing hypertension.
   - Manage stress: Practice stress-reducing techniques such as meditation, yoga, or deep breathing exercises.

2. Medic